In [ ]:

!pip install langchain==0.0.244
!pip install html2text==2020.1.16
!pip install google-api-core==2.11.1
!pip install google-api-python-client==2.95.0
!pip install google-auth==2.22.0
!pip install google-auth-httplib2==0.1.0
!pip install googleapis-common-protos==1.59.1
!pip install tiktoken==0.4.0
!pip install sentence_transformers
!pip install langchain_community
!pip install langchain_groq
!pip install -U langchain-qdrant
!pip install -U langchain-google-community

  Using cached langchain-0.0.244-py3-none-any.whl (1.4 MB)
  Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.60
    Uninstalling langsmith-0.1.60:
      Successfully uninstalled langsmith-0.1.60
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.0
    Uninstalling langchain-0.2.0:
      Successfully uninstalled langchain-0.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.0 requires langchain<0.3.0,>=0.2.0, but you have langchain 0.0.244 which is incompatible.
langchain-community 0.2.0 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-core 0.2.1 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
  Using cached openai-0.27.8-py3-none-any.w

  Using cached google_api_python_client-2.95.0-py2.py3-none-any.whl (11.5 MB)
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.129.0
    Uninstalling google-api-python-client-2.129.0:
      Successfully uninstalled google-api-python-client-2.129.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-community 1.0.4 requires google-api-core<3.0.0,>=2.17.1, but you have google-api-core 2.11.1 which is incompatible.
langchain-google-community 1.0.4 requires google-api-python-client<3.0.0,>=2.122.0, but you have google-api-python-client 2.95.0 which is incompatible.
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.2.0
    Uninstalling google-auth-httplib2-0.2.0:
      Succ

In [ ]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.vectorstores import Qdrant
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_groq import ChatGroq
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import Qdrant
from qdrant_client import QdrantClient
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_groq import ChatGroq
from qdrant_client.http import models
from google.colab import userdata
from langchain_google_community import GoogleSearchAPIWrapper
import logging

In [ ]:

# Initialize embedding function
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



# Initialize Qdrant client with in-memory storage
client = QdrantClient(":memory:")
collection_name = "langchain-test"

# Try creating the collection with appropriate distance metric
collections = client.get_collections()


# only create collection if it doesn't exist
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
collections = client.get_collections()
print(collections)

# Create vectorstore with the embeddings instance
vectorstore = Qdrant(
    client=client,
    collection_name=collection_name,
    embeddings=embedding_function
)
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize LLM
# No need to recreate the collection here, it's already handled
# LLM (ensure the API key and model name are correctly defined)
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


collections=[]
collections=[CollectionDescription(name='langchain-test')]


<ipython-input-4-5ac9a0a690d9>:24: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [ ]:
google_api_key=userdata.get('GOOGLE_API_KEY')
google_cse_id=userdata.get('GOOGLE_CSE_ID')

In [ ]:

search = GoogleSearchAPIWrapper(google_api_key=google_api_key ,google_cse_id=google_cse_id)

In [ ]:
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

In [ ]:


logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
from langchain.chains import RetrievalQAWithSourcesChain


qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="map_reduce",  # Adjust the chain type as necessary
    retriever=web_research_retriever,
    return_source_documents=True  # Ensure sources are returned
)
user_input = "find software developer jobs in hyderabad"
result = qa_chain({"question": user_input})
print(result)


Fetching pages: 100%|##########| 2/2 [00:01<00:00,  1.75it/s]


{'question': 'find software developer jobs in hyderabad', 'answer': "I don't know. The provided text does not contain any information about software developer jobs in Hyderabad.\n\n", 'sources': 'All sources provided.', 'source_documents': [Document(page_content='#  42,000+ Software Engineer Jobs in Greater Bengaluru Area\n\n  * Software Engineer, Search \n\n###  Software Engineer, Search\n\n####  Google\n\nBengaluru, Karnataka, India\n\nActively Hiring\n\n2 months ago\n\n  * Software Engineer, Core \n\n###  Software Engineer, Core\n\n####  Google\n\nBengaluru, Karnataka, India\n\nActively Hiring\n\n2 months ago\n\n  * SDE 1 - Backend \n\n###  SDE 1 - Backend\n\n####  Jar\n\nBengaluru, Karnataka, India\n\nActively Hiring\n\n5 months ago\n\n  * Software Engineer \n\n###  Software Engineer\n\n####  Microsoft\n\nBengaluru, Karnataka, India\n\nActively Hiring\n\n2 weeks ago\n\n  * Software Engineer \n\n###  Software Engineer\n\n####  Microsoft\n\nBengaluru, Karnataka, India\n\nActively Hir